# Distances API - Alternative Routes Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/DistancesAPI/AlternativeRoutes.ipynb)

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Subscribe to the Distances API at https://apis.signalocean.com/ to get a subscription key. Use the key to configure your connection to the API:

In [ ]:
pip install signal-ocean

In [ ]:
pip install folium

In [1]:
from signal_ocean import Connection

subscription_key = "" # paste your subscription key here
connection = Connection(subscription_key)

## Get a route and its alternatives
To get a route along with all possible alternative routes, use the `get_generic_point_to_point_route` method and set the `get_alternatives` parameter to `True`. For example, to find all possible routes between the ports in Milazzo and Basrah:

In [2]:
from decimal import Decimal
from signal_ocean.distances import DistancesAPI, Point

distances_api = DistancesAPI(connection)

milazzo_coordinates = Point(Decimal("38.2115"), Decimal("15.2665"))
basrah_coordinates = Point(Decimal("29.6964"), Decimal("48.8095"))

main_route = distances_api.get_generic_point_to_point_route(milazzo_coordinates, basrah_coordinates, get_alternatives=True)

The return value is a route that contains a list of its alternatives:

In [3]:
print("Main route distance:", main_route.distance)
print("Alternative routes:", len(main_route.alternative_paths))
for n, alternative in enumerate(main_route.alternative_paths, start=1):
    print(f"Alternative {n} distance: {alternative.distance}")

Main route distance: 4291.93
Alternative routes: 2
Alternative 1 distance: 4581.4
Alternative 2 distance: 11268.02


## Visualize the routes
To display the routes on a map:

In [4]:
import folium

m = folium.Map()
folium.Marker([milazzo_coordinates.lat, milazzo_coordinates.lon], popup="Milazzo").add_to(m)
folium.Marker([basrah_coordinates.lat, basrah_coordinates.lon], popup="Basrah").add_to(m)

all_routes = [main_route.calculated_route] + [alternative.calculated_route for alternative in main_route.alternative_paths]
colors = ["red", "blue", "green", "purple", "orange"]

for index, route in enumerate(all_routes):
    folium.PolyLine([[float(point.lat), float(point.lon)] for point in route], color=colors[index % len(colors)]).add_to(m)

latitudes = [float(point.lat) for point in route for route in all_routes]
longitudes = [float(point.lon) for point in route for route in all_routes]
m.fit_bounds([
    [min(latitudes), min(longitudes)],
    [max(latitudes), max(longitudes)],
])

m